# Milestone Project 2 - Blackjack Game

## Problem Statement

Create a Complete BlackJack Card Game in Python.

### Requirements

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

### Implementation Style

* OOP and classes in Python

## Design Process

### Game Card
* A game card is printed with a box dash. The card rank is printed as well
* All dealer's cards are hidden except the first card

### Classes, Methods, global Variables
* Create global variables to track:
  * current player
  * card suit, rank and Value
* Create a Card, Deck, and Bet classes

### Card Values
* All face cards (**Jack (J), Queen (Q), King (K)**) count as 10.
* Ace counts as 1 or 11 according to player choice
  * An ace can turn to 1 when
    * the game winner needs to be determined and a player sum exceeds 21 but has an ace among their cards
      * In this case, if the sum will be less than 21 with ace  becomes 1. 
      * This is a valid operation and they player has therefore not **BUST**
* All other cards count for their number 2 to 10.

### Determine number of decks to play with
  
### Player Options
* A player has the following options:
  * Split
    * A split occurs if the first two cards have the same value.
    * The player has the option to split or perform other actions
    * If a player decides to split
      * The two cards are split into separate bets
      * Each split has its own bet and the player can perform independent actions on each split
    * Note:
      * Multiple split can occur within a split 
  * Stand
    * A player **Stand** means the player has decided to take no more cards.
      * Note:
        * When a player stands, the dealer must reveal their cards.
          * If the dealer cannot take a stand yet, the dealer must continue to hit until the dealer hits a stand
            * This means the sum of dealers can must be at least 17.
  * Hit
    * If a player decides to **HIT**, the player must add an extra card to their cards
      * Note:
        * If a player card sum exceeds 21 after a **HIT**, that player has BUST. 
          * This means the dealer automatically wins the round!
  * Double
    * If a player decides to Double, this means the player doubles their wager
    * Next, the player must **HIT**. 
    * Finally, the game condition will be checked and a winner will be determined.
      * Note:
        * **Dealer Stand** must take effect. 

### Game States
* BlackJack
  * If a player has a total of 21 from the first two cards, that is a **BlackJack**.
    * This means this player has automatically won the round
* Push
  * A push occurs when the dealer and player have the total card sum
  * The player and dealer gets to keep their wager
* Bust
  * A bust occurs when one of the players exceed 21
  * A bust occurs for a player has a higher score than the dealer 
  * Loser loses their bet.
* Win
  * A win occurs when a player has a lower card sum than the dealer
  * A win can also occur if a dealer or player **BUST**
  * The winners get to keep the total wager
  * Note:
    * If both players exceed a total of 21, the player with the lower score gets 
* Dealer Stand
  * A dealer must relieve their hidden cards when a player decided to **STAND**.
  * Note:
    * After the dealer relieves their hidden cards,
    * If the dealer's sum is less than 17
      * the dealer must hit until a sum of at least 17 has been reached 
    * Once this condition is satisfied:
      * Check game condition and determine the winner


### Continue Games
* If a game status has been determined, Ask player if they would like to continue the game?

### Reset game
- Reset deck